## **TABLAS**
1. proveedores
* proveedor_id (INT, PRIMARY KEY, AUTO_INCREMENT)
* nombre (TEXT)
* direccion (TEXT)
* ciudad (TEXT)
* provincia (TEXT)
* codigo_proveedor (TEXT, UNIQUE)

2. categorias
* categoria_id (INT, PRIMARY KEY, AUTO_INCREMENT)
* nombre (TEXT)
* codigo_categoria (TEXT, UNIQUE)

3. piezas
* pieza_id (INT, PRIMARY KEY, AUTO_INCREMENT)
* codigo_pieza (TEXT, UNIQUE)
* nombre (TEXT)
* color (TEXT)
* precio (FLOAT)
* categoria_id (INT, FOREIGN KEY, REF: categorias(categoria_id))

4. suministros
* suministro_id (INT, PRIMARY KEY, AUTO_INCREMENT)
* proveedor_id (INT, FOREIGN KEY, REF: proveedores(proveedor_id))
* pieza_id (INT, FOREIGN KEY, REF: piezas(pieza_id))
* fecha (DATE)
* cantidad (INT)

## **RELACIONES**

1. Proveedores pueden suministrar varias piezas en diferentes fechas, asi que existe una relacion ***1 to many*** between **proveedores** y **suministros**.

2. Una pieza puede ser suministrada por varios proveedores, asi que existe una relacion ***1 to many*** entre **piezas** y **suministros**.
Un proveedor puede suministrar varias piezas también --> relación **N:M**

3. Cada pieza pertenece a una categoria, asi que existe una relacion ***1 to many*** entre **categorias** y **piezas**.

4. Suministros **N:M**

Tablas de proveedores y categorías quedan en los extremos, suministros y piezas quedan en medio.


In [22]:
import pandas as pd
import sqlite3
from datetime import date

In [23]:
# Connection and functions

conn = sqlite3.connect('./data/proveedores_piezas.db')
cursor = conn.cursor()

sqlq = lambda query, connection = conn: pd.read_sql(query, connection)

In [24]:
cursor.execute('SELECT name FROM sqlite_master WHERE type="table" AND name != "sqlite_sequence"')
tables = cursor.fetchall()

for table_name in tables:
    cursor.execute(f'DROP TABLE IF EXISTS {table_name[0]}')

conn.commit()
# conn.close()

In [25]:
sqlq('SELECT name FROM sqlite_master Where type="table"')

,name
0,sqlite_sequence


### 1. Creación de tablas

In [26]:
cursor.execute(''' 
CREATE TABLE IF NOT EXISTS proveedores (
    proveedor_id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL,
    direccion TEXT,
    ciudad TEXT,
    provincia TEXT,
    codigo_proveedor TEXT UNIQUE NOT NULL
)
''')

In [27]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS suministros (
    suminstros_id INTEGER PRIMARY KEY AUTOINCREMENT,
    proveedor_id INTEGER,
    pieza_id INTEGER,
    fecha DATETIME,
    cantidad INTEGER NOT NULL,
    FOREIGN KEY (proveedor_id) REFERENCES proveedores(proveedor_id)
    FOREIGN KEY (pieza_id) REFERENCES piezas(pieza_id) 
)
''')

In [28]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS piezas (
    pieza_id INTEGER PRIMARY KEY AUTOINCREMENT,
    codigo_pieza TEXT UNIQUE NOT NULL,
    nombre TEXT NOT NULL,
    color TEXT,
    precio FLOAT,
    categoria_id INTEGER,
    FOREIGN KEY (categoria_id) REFERENCES categorias(categoria_id)
)
''')

In [29]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS categorias (
    categoria_id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL,
    codigo_categoria TEXT UNIQUE NOT NULL
)
''')

### 2. Inlcusión de datos en tablas

In [30]:
prov_data = [
    ('Proveedor A', 'Calle Madrid', 'Madrid', 'Madrid', 'PROV1'),
    ('Proveedor B', 'Calle Barna', 'Barcelona', 'Barcelona', 'PROV2'),
    ('Proveedor C', 'Calle San Pío X', 'Valencia', 'Valencia', 'PROV3'),
    ('Proveedor D', 'Calle Estafeta', 'Pamplona', 'Navarra', 'PROV4'),
    ('Proveedor E', 'Calle Laurel', 'Logroño', 'Logroño', 'PROV5')
]
cursor.executemany(''' 
INSERT INTO proveedores (nombre, direccion, ciudad, provincia, codigo_proveedor)
VALUES (?,?,?,?,?) -- Marcador de posición para los valores que serán insertados en la tabla 
''', prov_data)

In [31]:
cat_data = [
    ('Vino', '001'),
    ('Distillado', '002')
]
cursor.executemany('''
INSERT INTO categorias (nombre, codigo_categoria)
VALUES (?,?) -- Marcador de posición para los valores que serán insertados en la tabla 
''', cat_data)

In [32]:
pieza_data = [
    ('P1', 'Sotovelo', 'Blanco', 20.50, '001'),
    ('P2', '10 Mil Botellas', 'Ambar', 80.70, '002'),
    ('P3', 'Domus Aurea', 'Tinto', 15.50, '001'),
    ('P4', 'Mar de Frades', 'Blanco', 20.50, '002'),
    ('P5', 'Ron Havana Selección de Maestros', 'Ambar', 65.50, '001')

]
cursor.executemany(''' 
INSERT INTO piezas (codigo_pieza, nombre, color, precio, categoria_id)
VALUES (?,?,?,?,?) -- Marcador de posición para los valores que serán insertados en la tabla 
''', pieza_data)

In [33]:
sumin_data = [
    (1, 1, '2024-07-01', 120),
    (2, 2, '2024-07-02', 30),
    (3, 3, '2024-08-05', 50),
    (4, 3, '2024-07-09', 20),
    (5, 5, '2024-12-07', 100),
    (5, 4, '2024-06-05', 120),    
    (5, 2, '2024-08-01', 150),
    (5, 1, '2024-11-02', 80),
    (5, 2, '2024-10-10', 90)    
]
cursor.executemany('''
INSERT INTO suministros (proveedor_id, pieza_id, fecha, cantidad)
VALUES (?,?,?,?) -- Marcador de posición para los valores que serán insertados en la tabla 
''', sumin_data)
conn.commit()

In [34]:
sqlq('SELECT * FROM sqlite_master WHERE type == "table"')

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,proveedores,proveedores,2,CREATE TABLE proveedores (\n proveedor_id I...
2,table,suministros,suministros,6,CREATE TABLE suministros (\n suminstros_id ...
3,table,piezas,piezas,7,CREATE TABLE piezas (\n pieza_id INTEGER PR...
4,table,categorias,categorias,9,CREATE TABLE categorias (\n categoria_id IN...


In [35]:
sqlq('SELECT * FROM proveedores')

,proveedor_id,nombre,direccion,ciudad,provincia,codigo_proveedor
0,1,Proveedor A,Calle Madrid,Madrid,Madrid,PROV1
1,2,Proveedor B,Calle Barna,Barcelona,Barcelona,PROV2
2,3,Proveedor C,Calle San Pío X,Valencia,Valencia,PROV3
3,4,Proveedor D,Calle Estafeta,Pamplona,Navarra,PROV4
4,5,Proveedor E,Calle Laurel,Logroño,Logroño,PROV5


In [36]:
sqlq('SELECT * FROM categorias')

,categoria_id,nombre,codigo_categoria
0,1,Vino,001
1,2,Distillado,002


In [37]:
sqlq('SELECT * FROM piezas')

,pieza_id,codigo_pieza,nombre,color,precio,categoria_id
0,1,P1,Sotovelo,Blanco,20.5,1
1,2,P2,10 Mil Botellas,Ambar,80.7,2
2,3,P3,Domus Aurea,Tinto,15.5,1
3,4,P4,Mar de Frades,Blanco,20.5,2
4,5,P5,Ron Havana Selección de Maestros,Ambar,65.5,1


In [38]:
sqlq('SELECT * FROM suministros')

,suminstros_id,proveedor_id,pieza_id,fecha,cantidad
0,1,1,1,2024-07-01,120
1,2,2,2,2024-07-02,30
2,3,3,3,2024-08-05,50
3,4,4,3,2024-07-09,20
4,5,5,5,2024-12-07,100
5,6,5,4,2024-06-05,120
6,7,5,2,2024-08-01,150
7,8,5,1,2024-11-02,80
8,9,5,2,2024-10-10,90


### 3. Demo queries

In [39]:
query = '''
SELECT 
        A.cantidad, A.fecha,
        B.nombre 'nombre_pieza', B.precio,
        C.nombre,
        D.codigo_proveedor
        
FROM suministros AS A
JOIN piezas AS B ON A.pieza_id = B.pieza_id
JOIN categorias AS C ON B.categoria_id = C.categoria_id
JOIN proveedores AS D ON A.proveedor_id = D.proveedor_id

WHERE nombre_pieza LIKE "Mar de Frades"
OR precio > 50
'''

sqlq(query)

,cantidad,fecha,nombre_pieza,precio,nombre,codigo_proveedor
0,30,2024-07-02,10 Mil Botellas,80.7,Distillado,PROV2
1,100,2024-12-07,Ron Havana Selección de Maestros,65.5,Vino,PROV5
2,120,2024-06-05,Mar de Frades,20.5,Distillado,PROV5
3,150,2024-08-01,10 Mil Botellas,80.7,Distillado,PROV5
4,90,2024-10-10,10 Mil Botellas,80.7,Distillado,PROV5


In [40]:
query = '''
SELECT 
        A.cantidad, A.fecha,
        B.nombre, B.precio
        
FROM suministros AS A
JOIN piezas AS B ON A.pieza_id = B.pieza_id

WHERE (B.nombre LIKE "Domus Aurea")
OR (A.cantidad > 80)
'''

sqlq(query)

,cantidad,fecha,nombre,precio
0,120,2024-07-01,Sotovelo,20.5
1,50,2024-08-05,Domus Aurea,15.5
2,20,2024-07-09,Domus Aurea,15.5
3,100,2024-12-07,Ron Havana Selección de Maestros,65.5
4,120,2024-06-05,Mar de Frades,20.5
5,150,2024-08-01,10 Mil Botellas,80.7
6,90,2024-10-10,10 Mil Botellas,80.7


In [41]:
conn.commit()
conn.close()